In [2]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.2 MB/s 
     |████████████████████████████████| 57 kB 4.8 MB/s 
     |████████████████████████████████| 112 kB 55.1 MB/s 
     |████████████████████████████████| 212 kB 46.8 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 270 kB 49.7 MB/s 
     |████████████████████████████████| 84 kB 3.2 MB/s 
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 84 kB 2.8 MB/s 
     |████████████████████████████████| 2.3 MB 37.5 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 80 kB 8.3 MB/s 
     |████████████████████████████████| 68 kB 4.5 MB/s 
     |████████████████████████████████| 46 kB 3.5 MB/s 
     |████████████████████████████████| 856 kB 72.7 MB/s 
     |████████████████████████████████| 593 kB 40.

In [3]:
import gradio as gr
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext
from torchtext import data


In [11]:
class BaselineModel(torch.nn.Module):
    def __init__(self, vocab):
        super().__init__()
        # initialize word vectors to random numbers 
        self.embedding = torch.nn.Embedding.from_pretrained(vocab.vectors)
        self.out=torch.nn.Linear(100,1)
        #TO DO
        
        # prediction function takes embedding as input, and predicts which word in vocabulary as output

        #TO DO
        
    def forward(self, x):
        """
        x: torch.tensor of shape (bsz), bsz is the batch size
        """
        #TO DO
        e=0
        for item in x:
          e+=self.embedding(item)
        e/= len(x)
        logits=self.out(e).squeeze()
        #out=torch.nn.functional.sigmoid(logits).squeeze()
        
        return logits

# cnn model
class CNNModel(torch.nn.Module): # CNN model
    def __init__(self, vocab, n1,n2, k1,k2):
        super().__init__()
        # initialize word vectors to random numbers 
        self.embedding = torch.nn.Embedding.from_pretrained(vocab.vectors)
        self.conv1=torch.nn.Conv2d(1,n1,(100,k1),bias=False)
        self.conv2=torch.nn.Conv2d(1,n2,(100,k2),bias=False)
        self.activate=torch.nn.ReLU()
        self.maxpool= torch.nn.MaxPool2d((1,500),ceil_mode=True)
        #self.maxpool2= torch.nn.MaxPool1d()
        self.dropout = torch.nn.Dropout(0.25)
        self.out= torch.nn.Linear(n1+n2,1)
        #TO DO
        
        # prediction function takes embedding as input, and predicts which word in vocabulary as output

        #TO DO
        
    def forward(self, x):
        """
        x: torch.tensor of shape (bsz), bsz is the batch size
        """
        #TO DO
        #print(x.size())
        embed=self.embedding(x)
        #print(embed.size())
        embed=embed.T.unsqueeze(0)
        #print(embed.size())
        conv1=self.conv1(embed)
        conv1=self.activate(conv1)
        conv2=self.conv2(embed)
        conv2=self.activate(conv2)
        #conv1=conv1.squeeze()
        #conv2=conv2.squeeze()
        #print(conv1.size())
        #print(conv2.size())
        max1= self.maxpool(conv1)
        #print(max1.size())
        max2=self.maxpool(conv2)
        #print(max2.size())
        #drop=self.dropout(torch.cat((max1,max2)).squeeze())
        #output=self.out(drop)
        output=self.out(torch.cat((max1,max2)).squeeze())
        return output

glove = torchtext.vocab.GloVe(name="6B",dim=100)

In [12]:
# load baseline model from saved checkpoint
baseline_model = BaselineModel(glove)
baseline_model.load_state_dict(torch.load('/content/baseline.pt'))
baseline_model.eval()

# load cnn model from saved checkpoint
cnn_model = CNNModel(glove,25,25,2,3)
cnn_model.load_state_dict(torch.load('/content/CNN.pt', map_location=torch.device('cpu')))
cnn_model.eval()



CNNModel(
  (embedding): Embedding(400000, 100)
  (conv1): Conv2d(1, 25, kernel_size=(100, 2), stride=(1, 1), bias=False)
  (conv2): Conv2d(1, 25, kernel_size=(100, 3), stride=(1, 1), bias=False)
  (activate): ReLU()
  (maxpool): MaxPool2d(kernel_size=(1, 500), stride=(1, 500), padding=0, dilation=1, ceil_mode=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (out): Linear(in_features=50, out_features=1, bias=True)
)

In [13]:
# baseline prediction function with output probability and label
def baseline_predict(sentence):
	tokens = sentence.split()
	# Convert to integer representation per token
	token_ints = [glove.stoi.get(tok, len(glove.stoi)-1) for tok in tokens]
	# Convert into a tensor of the shape accepted by the models
	token_tensor = torch.LongTensor(token_ints).view(-1,1)
	# Get the output from the model
	prediction = baseline_model(token_tensor)
	prediction = torch.sigmoid(prediction)
	prediction = prediction.item()
	if prediction > 0.5:
		return prediction, "Subjective"
	else:
		return prediction, "Objective"

# cnn prediction function with output probability and label
def cnn_predict(sentence):
	tokens = sentence.split()
	# Convert to integer representation per token
	token_ints = [glove.stoi.get(tok, len(glove.stoi)-1) for tok in tokens]
	# Convert into a tensor of the shape accepted by the models
	token_tensor = torch.LongTensor(token_ints).view(-1,1)
	# Get the output from the model
	prediction = cnn_model(token_tensor.flatten())
	prediction = torch.sigmoid(prediction)
	prediction = prediction.item()
	if prediction > 0.5:
		return prediction, "Subjective"
	else:
		return prediction, "Objective"

# baseline interface
baseline_interface = gr.Interface(fn=baseline_predict, inputs="text", outputs= [gr.outputs.Textbox(label="Baseline Probability"), gr.outputs.Textbox(label="Baseline Label")], title="Baseline Model", description="Enter a sentence to predict whether it is subjective or objective", allow_flagging=False)


# cnn interface
cnn_interface = gr.Interface(fn=cnn_predict, inputs="text", outputs=[gr.outputs.Textbox(label="CNN Probability"), gr.outputs.Textbox(label="CNN Label")], title="CNN Model", description="Enter a sentence to predict whether it is subjective or objective.", allow_flagging=False)


gr.Parallel(baseline_interface, cnn_interface).launch(debug=True)

/usr/local/lib/python3.7/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  "Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components",
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/interface.py:356: UserWarning: The `allow_flagging` parameter in `Interface` nowtakes a string value ('auto', 'manual', or 'never'), not a boolean. Setting parameter to: 'never'.
  "The `allow_flagging` parameter in `Interface` now"


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://19556.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7f91e027f590>,
 'http://127.0.0.1:7861/',
 'https://19556.gradio.app')